In [1]:
from hearlight import HearLight

In [2]:
dashboard = HearLight()

In [3]:
dashboard.main_control_panel

Box(children=(HTML(value='\n<style>\n    .logo_panel_css{\n        height : 330px;\n        width : 100%;\n   …

In [4]:
dashboard.led_control_panel

Box(children=(HTML(value='\n<style>\n    .logo_panel_css{\n        height : 330px;\n        width : 100%;\n   …

In [5]:
dashboard.required_currents_panel

In [6]:
dashboard.channel_currents_panel

In [7]:
dashboard.actual_irradiances_panel

In [8]:
dashboard.logo_panel

In [15]:
# html = f"""
# <style>
#     .btn_width{{
#         width : calc(200px + 100px);
#     }}
# </style>
# """

# import ipywidgets as widgets
# btn = widgets.Button()
# btn.add_class('btn_width')

# widgets.Box(children=[widgets.HTML(value=html), btn])

In [5]:
# import asyncio

# # start background task to allow user to work while grid is loading
# select_led_grid_gen_task = asyncio.ensure_future(dashboard.select_leds.get_led_grid())
# indicate_led_grid_gen_task = asyncio.ensure_future(dashboard.indicate_leds.get_led_grid())

In [12]:
#from ipysheet import sheet, cell, row, column, cell_range
# import ipysheet
# import numpy as np

In [13]:
# required_currents = np.asarray([[1,2,3],[4,5,6],[7,8,9]])

In [14]:
# required_currents

In [15]:
# req_currents_display = ipysheet.sheet(rows=3, columns=3)
# cell1 = ipysheet.cell(0,0,0)

In [16]:
# ipysheet.sheet(0,0)

In [6]:
# req_currents_display

In [5]:
# control_array.irr_to_current_upload

In [ ]:
# control_array.current_to_irr_upload

In [ ]:
# # TEST READING COEFFICIENT FILES AND APPLYING REGRESSION

# # open file and read irradiance to current regression coefficients
# data_str = control_array.irr_to_current_upload.data[0].decode("utf-8").split('\n')[0:-1]

# #
# #
# #
# irr_test = 15
# led_max_current = 100
# channel_max_current = 100
# switch_max_current = 130
# device_max_current = 400
# #
# #
# #

# # get regression coefficients as 2D numpy array
# polynomial_order = len(data_str[0].split(',')) - 1
# coeffs_matrix = np.reshape(np.asarray([d.split(',') for d in data_str]).astype(float), (10,10,polynomial_order+1))

# if irr_test == 0 or np.array_equal(control_array.indicate_leds.matrix_output, np.zeros(shape=(N_SWITCHES, N_CHANNELS))):
#     required_currents = np.zeros(shape=(N_SWITCHES, N_CHANNELS), dtype=float)
#     channel_currents = np.zeros(shape=(1,N_CHANNELS), dtype=float)
#     actual_led_irradiances = np.zeros(shape=(N_SWITCHES, N_CHANNELS), dtype=float)
    
# else:
#     # get matrix of required currents
#     irr_matrix = np.array([[irr_test] * N_CHANNELS for _ in range(N_SWITCHES)])
#     required_currents = np.asarray([[np.poly1d(coeffs)(irr_test) for coeffs in coeffs_matrix[r]] for r in range(N_SWITCHES)]) * control_array.indicate_leds.matrix_output

#     # check if individual LED maximum current is being exceeded and set to maximum if so
#     required_currents[required_currents > led_max_current] = led_max_current

#     # get required channel currents from matrix of required currents unless required currents all zero
#     channel_currents = np.zeros(shape=(1,N_CHANNELS), dtype=float)
#     if not np.array_equal(required_currents, np.zeros(shape=(N_SWITCHES, N_CHANNELS))):
#         n_switches_closed = control_array.switch_states[0].sum()
#         channel_currents[0][control_array.channel_states[0]] = np.amin(required_currents[control_array.switch_states[0],:][:,control_array.channel_states[0]], axis=0) * n_switches_closed

#     # check if channel current exceeded for any channel
#     channel_currents[0][channel_currents[0] > channel_max_current] = channel_max_current
    
#     # check if any switch current exceeded
#     #
#     # ADD WARNING MESSAGES HERE!
#     #
#     requested_switch_current = np.sum(channel_currents[0]) / n_switches_closed
#     if requested_switch_current > switch_max_current:
#         channel_currents[0] = channel_currents[0] / requested_switch_current * switch_max_current
        
#     # check if maximum device current exceeded
#     requested_device_current = np.sum(channel_currents[0])
#     if requested_device_current > device_max_current:
#         channel_currents[0] = channel_currents[0] / requested_device_current * device_max_current
        
#     # implement regression to display ACTUAL irradiances based on the supplied current
#     data_str = control_array.current_to_irr_upload.data[0].decode("utf-8").split('\n')[0:-1]
    
#     polynomial_order = len(data_str[0].split(',')) - 1
#     coeffs_matrix = np.reshape(np.asarray([d.split(',') for d in data_str]).astype(float), (10,10,polynomial_order+1))
#     actual_led_irradiances = np.asarray([[np.poly1d(coeffs_matrix[r][c])(channel_currents[0][c]/n_switches_closed) for c in range(N_CHANNELS)] for r in range(N_SWITCHES)]) * control_array.indicate_leds.matrix_output

In [ ]:
#channel_currents

In [ ]:
#actual_led_irradiances